In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv

План:
1- Выяснить кол-во страниц
2- Сформировать список урлов
3- Собрать данные

In [2]:
def get_html(url):
    r = requests.get(url)
    return r.text

In [3]:
def get_total_pages(html):
    soup = BeautifulSoup(html, 'lxml')
    
    # Если номера всех страниц умещаются, то кол-во страниц будет неперное (Исправлено)
    total_pages = int(soup.find('div', class_='pagenav').find_all('a', class_='smallfont')[-1].get('href').split('=')[-1])
    
    if len(soup.find('div', class_='pagenav').find_all('a', class_='smallfont')) > total_pages:
        total_pages = len(soup.find('div', class_='pagenav').find_all('a', class_='smallfont'))
    return total_pages

In [4]:
def get_page_data(url_digital, answers, views):
    
    url_page_part = 'http://forum.watch.ru/showthread.php?t='
    url = url_page_part + url_digital
    
    ad_page = get_html(url)
    soup = BeautifulSoup(ad_page, 'lxml')
    
    main_soup = soup.find('td', class_='alt1', id=True).find('div', class_='messagebody')
    
    try:
        # ad title - заголовок объявления
        ad_title = soup.find('td', class_='alt1').find('td', class_='navbar').find('strong').text.strip()
    except:
        ad_title = ''
        
    try:
        # name - название товара
        name = main_soup.find('b').text.strip()
    except:
        name = ''
        
    try:
        # sex - для кого часы по полу
        sex = re.findall(r'Пол:\s*([^\n]*)', main_soup.text, flags=re.I)[0].strip()
    except:
        sex = ''
        
    try:
        # year - Год покупки
        year = re.findall(r'покупки:\s*([^\n]*)', main_soup.text, flags=re.I)[0].strip()
    except:
        year = ''
     
    try:
        # condition - состояние(N - новые)
        condition = re.findall(r'состояние:\s*([^\(]*)', main_soup.text, flags=re.I)[0].strip()
    except:
        condition = ''    
        
    try:
        # polish - полировка часов(есть, нет)
        if main_soup.text.find('состояние: не указано') == -1: 
            polish = re.findall(r'Полировка:\s*([^\n]*)', main_soup.text, flags=re.I)[0].strip()
        else:
            polish = condition = re.findall(r'состояние:\s*([^\n]*)', main_soup.text, flags=re.I)[0].strip()
    except:
        polish = ''  

    try:
        # price - цена(руб, евро, дол)
        price = re.findall(r'Цена:\s*([^\(n)]*)', main_soup.text, flags=re.I)[0].strip()
    except:
        price = ''  
        
    try:
        # box - наличие коробки от часов
        box = re.findall(r'Коробка:\s*([^\n]*)', main_soup.text, flags=re.I)[0].strip()
    except:
        box = ''
        
    try:
        # documentation - наличике документов
        documentation = re.findall(r'Документы:\s*([^\n]*)', main_soup.text, flags=re.I)[0].strip()
    except:
        documentation = ''
        
    try:
        # case_diameter - Диаметр корпуса
        case_diameter = re.findall(r'Диаметр корпуса:\s*([^\n]*)', main_soup.text, flags=re.I)[0].strip()
    except:
        case_diameter = ''
 
    try:
        # movement - тип механизма
        movement = re.findall(r'Механизм:\s*([^\n]*)', main_soup.text, flags=re.I)[0].strip()
    except:
        movement = ''
        
    try:
        # reference - Референс
        reference = re.findall(r'Референс:\s*([^\n]*)', main_soup.text, flags=re.I)[0].strip()
    except:
        reference = ''
        
    try:
        # function - Функции часов
        function = re.findall(r'Функции:\s*([^\n]*)', main_soup.text, flags=re.I)[0].strip()
    except:
        function = ''
        
    try:
        # manufacturer_price - Цена, рекомендованная производителем:
        manufacturer_price = re.findall(r'Цена, рекомендованная производителем:\s*([^\n]*)',\
                                        main_soup.text, flags=re.I)[0].strip()
    except:
        manufacturer_price = ''
        
    try:
        # data_of_purchase - дата подачи объявления
        data_of_purchase = soup.find_all('td', class_='thead')[1].find_all('div', class_='normal')[-1].text.strip()
    except:
        data_of_purchase = ''
        
    try:
        # item_status - статус объявления(В ПРОДАЖЕ, ПРОДАНО)
        item_status = soup.find_all('span', class_='maamenu-statusbuyer marketstatuscolor_1')[0].text.strip()
    except:
        item_status = ''
        
    try:
        # time_item_status - время послежнего изменения в статусе объявления
        time_item_status_bs = soup.find_all('span', class_='smallfont time')[0].text.strip()
        time_item_status = re.findall(r'изменен\s*([^\)]*)', time_item_status_bs, flags=re.I)[0].strip()
    except:
        time_item_status = ''
        
    try:
        # login - логин продовца
        login = soup.find_all('a', class_='bigusername')[0].text.strip()
    except:
        login = ''
        
    try:
        # login_forum_status - статус продовца на форуме(новичок, Форумчанин)
        login_forum_status = soup.find_all('div', class_='smallfont')[2].text.strip()
    except:
        login_forum_status = ''
        
    if views.find(',') != -1:
        views = views.split(',')[0] + views.split(',')[1]
        
    if answers.find(',') != -1:
        answers = answers.split(',')[0] + answers.split(',')[1]
    
    data = {'ad_id': url_digital,
            'ad_title': ad_title,
            'name': name,
            'data_of_purchase': data_of_purchase,
            'sex': sex,
            'year': year,
            'condition': condition,
            'polish': polish,
            'price': price,
            'box': box,
            'documentation': documentation,
            'case_diameter': case_diameter,
            'movement': movement,
            'reference': reference,
            'function': function,
            'manufacturer_price': manufacturer_price,
            'item_status': item_status,
            'time_item_status': time_item_status,
            'login': login,
            'login_forum_status': login_forum_status,
            'answers': answers,
            'views': views,
            'url': url}
    
    return data

In [ ]:
def get_page_url(html, url_number):
    soup = BeautifulSoup(html, 'lxml')
    
    ad_page_url = soup.find('tbody', id='threadbits_forum_' + str(url_number))

    url_all_page = []
    
    for i in range(len(ad_page_url.findAll('tr', id=True))):
        
        url_digit_views_answers = []
        
        try:
            id_digital = ad_page_url.findAll('tr', id=True)[i].find_all(href=True)[1].get('href').split('=')[-1]
            url_digit_views_answers.append(id_digital)
        except:
            pass
        
        # answers - кол-во ответов под объявлением
        url_digit_views_answers.append(ad_page_url.find_all('tr')[i].find_all('td')[-2].text)
        
        # views - кол-во просмотров объявления
        url_digit_views_answers.append(ad_page_url.find_all('tr')[i].find_all('td')[-1].text)
        
        url_all_page.append(url_digit_views_answers)
    
    return url_all_page

In [ ]:
def write_to_csv(data, i):
    with open('out_data/data_{}.csv'.format(i), 'a', encoding='utf-8') as f:
        writer = csv.writer(f)
        
        writer.writerow( (data['ad_id'],
                        data['ad_title'],
                        data['name'],
                        data['data_of_purchase'],
                        data['sex'],
                        data['year'],
                        data['condition'],
                        data['polish'],
                        data['price'],
                        data['box'],
                        data['documentation'],
                        data['case_diameter'],
                        data['movement'],
                        data['reference'],
                        data['function'],
                        data['manufacturer_price'],
                        data['item_status'],
                        data['time_item_status'],
                        data['login'],
                        data['login_forum_status'],
                        data['answers'],
                        data['views'],
                        data['url']) )


### Начало кода

In [ ]:
# url_set = ['http://forum.watch.ru/forumdisplay.php?f=192',\
#             'http://forum.watch.ru/forumdisplay.php?f=193',\
#             'http://forum.watch.ru/forumdisplay.php?f=130',\
#             'http://forum.watch.ru/forumdisplay.php?f=94',\
#             'http://forum.watch.ru/forumdisplay.php?f=131',\
#             'http://forum.watch.ru/forumdisplay.php?f=173',\
#             'http://forum.watch.ru/forumdisplay.php?f=174',\
#             'http://forum.watch.ru/forumdisplay.php?f=93']

# url_set = [ 'http://forum.watch.ru/forumdisplay.php?f=131',\
#             'http://forum.watch.ru/forumdisplay.php?f=173',\
#             'http://forum.watch.ru/forumdisplay.php?f=174',\
#             'http://forum.watch.ru/forumdisplay.php?f=93']

url_set = ['http://forum.watch.ru/forumdisplay.php?f=172']
for url in url_set:
    url_number = int(re.findall(r'f=\s*([^\\]*)', url, flags=re.I)[0].strip())
    
    page_part = '&page='
    total_pages = get_total_pages(get_html(url))

    url_list =[]
    
# Получаем url всех страниц в каталоге
for i in range(1, total_pages + 1):
    url_gen = url + page_part + str(i)
    url_list.append(url_gen)

In [ ]:
# Получаем номера, кол-во ответов, кол-во просмортров всех объявлений на одной странице 
url_digital_pages = []

start_page = 0
for i in range(start_page, len(url_list)):
    print('Номер страницы - ', i + 1, ' ', url_list[i])
    html_temp = get_page_url(get_html(url_list[i]), url_number)
    url_digital_pages = html_temp
        
    print('  Получаем номера, кол-во ответов, кол-во просмортров всех объявлений на одной странице ')
        
    # Тк рекалмные объявления висят на какждой странице, то их нужно оставить в единственном экземпляре
    url_digital_pages_set = []
    set_ = []
        
    for id in url_digital_pages:
        if id[0] in set_:
            pass
        else:
            set_.append(id[0])
            url_digital_pages_set.append(id)

    # Запускаем парсинг страниц
    print(' Запускаем парсинг страниц и запись ')
    for col in url_digital_pages_set:
        write_to_csv(get_page_data(col[0], col[1], col[2]), url_number)  

Номер страницы -  1   http://forum.watch.ru/forumdisplay.php?f=172&page=1
  Получаем номера, кол-во ответов, кол-во просмортров всех объявлений на одной странице 
 Запускаем парсинг страниц и запись 
Номер страницы -  2   http://forum.watch.ru/forumdisplay.php?f=172&page=2
  Получаем номера, кол-во ответов, кол-во просмортров всех объявлений на одной странице 
 Запускаем парсинг страниц и запись 
Номер страницы -  3   http://forum.watch.ru/forumdisplay.php?f=172&page=3
  Получаем номера, кол-во ответов, кол-во просмортров всех объявлений на одной странице 
 Запускаем парсинг страниц и запись 
Номер страницы -  4   http://forum.watch.ru/forumdisplay.php?f=172&page=4
  Получаем номера, кол-во ответов, кол-во просмортров всех объявлений на одной странице 
 Запускаем парсинг страниц и запись 
Номер страницы -  5   http://forum.watch.ru/forumdisplay.php?f=172&page=5
  Получаем номера, кол-во ответов, кол-во просмортров всех объявлений на одной странице 
 Запускаем парсинг страниц и запись 


Номер страницы -  42   http://forum.watch.ru/forumdisplay.php?f=172&page=42
  Получаем номера, кол-во ответов, кол-во просмортров всех объявлений на одной странице 
 Запускаем парсинг страниц и запись 
Номер страницы -  43   http://forum.watch.ru/forumdisplay.php?f=172&page=43
  Получаем номера, кол-во ответов, кол-во просмортров всех объявлений на одной странице 
 Запускаем парсинг страниц и запись 
Номер страницы -  44   http://forum.watch.ru/forumdisplay.php?f=172&page=44
  Получаем номера, кол-во ответов, кол-во просмортров всех объявлений на одной странице 
 Запускаем парсинг страниц и запись 
Номер страницы -  45   http://forum.watch.ru/forumdisplay.php?f=172&page=45
  Получаем номера, кол-во ответов, кол-во просмортров всех объявлений на одной странице 
 Запускаем парсинг страниц и запись 
Номер страницы -  46   http://forum.watch.ru/forumdisplay.php?f=172&page=46
  Получаем номера, кол-во ответов, кол-во просмортров всех объявлений на одной странице 
 Запускаем парсинг страниц 